In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

In [2]:
data = pd.read_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lineSuin.csv')
data
# 카페 데이터만
data = data.loc[data['상권업종중분류명'] == '커피점/카페']  
data

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,역명,거리,카카오url
3,182,25472959,노란고무줄,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135280,6198.0,NaN,1,NaN,127.050045,37.502899,분당 한티,547.173123,NaN
7,330,20296218,홈스테드,커피타워팰리스점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135270,6292.0,NaN,NaN,NaN,127.053198,37.488742,분당 도곡,243.138693,NaN
11,382,25390239,Manna,NaN,Q,음식,Q12,커피점/카페,Q12A03,생과일주스전문점,...,135744,6367.0,NaN,2,NaN,127.103094,37.487311,분당 수서,139.233515,NaN
12,397,25964875,스타벅스,한티점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135280,6206.0,NaN,6,NaN,127.053715,37.497946,분당 한티,149.558597,NaN
13,428,25363926,커피팩토리,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135010,6055.0,NaN,NaN,NaN,127.037622,37.521739,분당 강남구청,509.770567,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4578,135218,17252476,총알커피,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,133824,4768.0,NaN,NaN,NaN,127.044089,37.546850,분당 서울숲,204.699780,NaN
4592,135511,17196732,컴포즈커피,대치점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135840,6194.0,NaN,1,107,127.053512,37.505072,분당 한티,597.839979,NaN
4593,135523,17254647,스테이지246,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135832,6098.0,NaN,4,NaN,127.036546,37.514765,분당 강남구청,550.784894,NaN
4594,135525,17250772,카페,라스보드,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135896,6017.0,NaN,4,NaN,127.036670,37.527110,분당 압구정로데오,531.075407,NaN


In [3]:
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리","카카오url"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
3,노란고무줄,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 선릉로 424,분당 한티,547.173123,NaN,한티,한티역 노란고무줄
7,홈스테드,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 언주로30길 39,분당 도곡,243.138693,NaN,도곡,도곡역 홈스테드
11,Manna,커피점/카페,생과일주스전문점,서울특별시 강남구 광평로 280,분당 수서,139.233515,NaN,수서,수서역 Manna
12,스타벅스,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 선릉로64길 18,분당 한티,149.558597,NaN,한티,한티역 스타벅스
13,커피팩토리,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 도산대로50길 9,분당 강남구청,509.770567,NaN,강남구청,강남구청역 커피팩토리
...,...,...,...,...,...,...,...,...,...
4578,총알커피,커피점/카페,커피전문점/카페/다방,서울특별시 성동구 왕십리로5길 9-2,분당 서울숲,204.699780,NaN,서울숲,서울숲역 총알커피
4592,컴포즈커피,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 선릉로90길 56,분당 한티,597.839979,NaN,한티,한티역 컴포즈커피
4593,스테이지246,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 언주로130길 12,분당 강남구청,550.784894,NaN,강남구청,강남구청역 스테이지246
4594,카페,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 언주로172길 36,분당 압구정로데오,531.075407,NaN,압구정로데오,압구정로데오역 카페


In [4]:
items = data["검색어"][540:]
items

3029         압구정로데오역 아난티
3033             도곡역 톨릭스
3044    왕십리역 커피홀엔터식스한양대점
3045            수서역 스타벅스
3048         한티역 행복플러스카페
              ...       
4578           서울숲역 총알커피
4592           한티역 컴포즈커피
4593       강남구청역 스테이지246
4594          압구정로데오역 카페
4595           강남구청역 MMN
Name: 검색어, Length: 286, dtype: object

In [5]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}

In [6]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path='C:/Users/정예림/Desktop/추천시스템 프로젝트/chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 



count = 0
current = 0
indexing = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        #상세보기 URL수집
        url = driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]').get_attribute('href')
        data.iloc[indexing,-3] = url
        print(url)
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        time.sleep(20)
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('.link_photo > img') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_ur}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(1)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('.link_photo > img') #이미지url
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                        
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        data.iloc[indexing,-3] = None
        print("식당 존재 x")
    
    indexing += 1


진행상황 : 1/286
식당 존재
https://place.map.kakao.com/38295229
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2/286
식당 존재
https://place.map.kakao.com/17419369
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 3/286
식당 존재 x
진행상황 : 4/286
식당 존재
https://place.map.kakao.com/8438450
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 5/286
식당 존재 x
진행상황 : 6/286
식당 존재
https://place.map.kakao.com/27491518
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 7/286
식당 존재
https://place.map.kakao.com/1378562160
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 8/286
식당 존재 x
진행상황 : 9/286
식당 존재 x
진행상황 : 10/286
식당 존재 x
진행상황 : 11/286
식당 존재 x
진행상황 : 12/286
식당 존재
https://place.map.kakao.com/286483883
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 13/286
식당 존재
https://place.map.kakao.com/2108441068
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 14/286
식당 존재
https://place.map.kakao.com/1940830202
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 15/286
식당 존재
https://place.map.kakao.com/18197885
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 16/286
식당 존재
https://place.map.kakao.com/26809710
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상

식당 존재
https://place.map.kakao.com/8438450
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 178/286
식당 존재 x
진행상황 : 179/286
식당 존재
https://place.map.kakao.com/2089373601
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 180/286
식당 존재 x
진행상황 : 181/286
식당 존재
https://place.map.kakao.com/931848488
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 182/286
식당 존재
https://place.map.kakao.com/22219175
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 183/286
식당 존재
https://place.map.kakao.com/8103664
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 184/286
식당 존재 x
진행상황 : 185/286
식당 존재
https://place.map.kakao.com/14722215
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 186/286
식당 존재
https://place.map.kakao.com/1571972074
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 187/286
식당 존재
https://place.map.kakao.com/14864346
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 188/286
식당 존재 x
진행상황 : 189/286
식당 존재
https://place.map.kakao.com/1174842551
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 190/286
식당 존재
https://place.map.kakao.com/239253861
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 191/286
식당 존재
https://place.map.

In [7]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,1i768dg,도곡역 톨릭스,1점,2021.04.04.
1,n06bac,도곡역 톨릭스,2점,2021.02.21.
2,mueigh,도곡역 톨릭스,3점,2020.12.07.
3,vgsrbe,도곡역 톨릭스,4점,2020.09.14.
4,1a35re4,도곡역 톨릭스,5점,2020.07.22.
...,...,...,...,...
1432,None,압구정로데오역 카페,None,2018.10.21.
1433,None,압구정로데오역 카페,None,2018.08.04.
1434,None,압구정로데오역 카페,None,2018.07.19.
1435,n1rk9i,강남구청역 MMN,5점,2021.04.21.


In [8]:
img_elem

,ItemID,img_url
0,도곡역 톨릭스,None
1,도곡역 톨릭스,None
2,도곡역 톨릭스,None
3,도곡역 톨릭스,None
4,도곡역 톨릭스,None
...,...,...
1432,압구정로데오역 카페,None
1433,압구정로데오역 카페,[]
1434,압구정로데오역 카페,None
1435,강남구청역 MMN,None


In [9]:
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
3,노란고무줄,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 선릉로 424,분당 한티,547.173123,https://place.map.kakao.com/38295229,한티,한티역 노란고무줄
7,홈스테드,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 언주로30길 39,분당 도곡,243.138693,https://place.map.kakao.com/17419369,도곡,도곡역 홈스테드
11,Manna,커피점/카페,생과일주스전문점,서울특별시 강남구 광평로 280,분당 수서,139.233515,None,수서,수서역 Manna
12,스타벅스,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 선릉로64길 18,분당 한티,149.558597,https://place.map.kakao.com/8438450,한티,한티역 스타벅스
13,커피팩토리,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 도산대로50길 9,분당 강남구청,509.770567,None,강남구청,강남구청역 커피팩토리
...,...,...,...,...,...,...,...,...,...
4578,총알커피,커피점/카페,커피전문점/카페/다방,서울특별시 성동구 왕십리로5길 9-2,분당 서울숲,204.699780,NaN,서울숲,서울숲역 총알커피
4592,컴포즈커피,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 선릉로90길 56,분당 한티,597.839979,NaN,한티,한티역 컴포즈커피
4593,스테이지246,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 언주로130길 12,분당 강남구청,550.784894,NaN,강남구청,강남구청역 스테이지246
4594,카페,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 언주로172길 36,분당 압구정로데오,531.075407,NaN,압구정로데오,압구정로데오역 카페


In [10]:
img_elem

,ItemID,img_url
0,도곡역 톨릭스,None
1,도곡역 톨릭스,None
2,도곡역 톨릭스,None
3,도곡역 톨릭스,None
4,도곡역 톨릭스,None
...,...,...
1432,압구정로데오역 카페,None
1433,압구정로데오역 카페,[]
1434,압구정로데오역 카페,None
1435,강남구청역 MMN,None


In [11]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,1i768dg,도곡역 톨릭스,1점,2021.04.04.
1,n06bac,도곡역 톨릭스,2점,2021.02.21.
2,mueigh,도곡역 톨릭스,3점,2020.12.07.
3,vgsrbe,도곡역 톨릭스,4점,2020.09.14.
4,1a35re4,도곡역 톨릭스,5점,2020.07.22.
...,...,...,...,...
1432,None,압구정로데오역 카페,None,2018.10.21.
1433,None,압구정로데오역 카페,None,2018.08.04.
1434,None,압구정로데오역 카페,None,2018.07.19.
1435,n1rk9i,강남구청역 MMN,5점,2021.04.21.


In [12]:
review_elem

,ItemID,review
0,도곡역 톨릭스,들어가니 예약한다고해서 예약기계에 핸드폰번호 넣고 밖에서 기다렸습니다. 30분이 지...
1,도곡역 톨릭스,묘하게 급식 맛이 나요...
2,도곡역 톨릭스,무난무난... 동네랑은 잘 어울리는데 맛은 완전 평범해요 집에서 어렵지않게 해먹을 ...
3,도곡역 톨릭스,언제가도 맛있는곳
4,도곡역 톨릭스,최고예요
...,...,...
1432,압구정로데오역 카페,보통이에요
1433,압구정로데오역 카페,"케익, 예쁘고 맛있어요 ❤️"
1434,압구정로데오역 카페,최고예요
1435,강남구청역 MMN,최고예요


In [13]:
img_elem.head()

,ItemID,img_url
0,도곡역 톨릭스,None
1,도곡역 톨릭스,None
2,도곡역 톨릭스,None
3,도곡역 톨릭스,None
4,도곡역 톨릭스,None


In [14]:
ItemID_elem = rating_df['ItemID'].unique()

for k in range(len(ItemID_elem)) :
    User_ID_elem = rating_df.loc[rating_df['ItemID'] == ItemID_elem[k]]
    Uder_ID_elem = User_ID_elem[['UserID','ItemID']]
print(User_ID_elem)





ItemID_elem = img_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_img_elem = img_elem.loc[img_elem['ItemID'] == ItemID_elem[j]]
print(item_img_elem)



ItemID_elem = review_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_review_elem = review_elem.loc[review_elem['ItemID'] == ItemID_elem[j]]
print(item_review_elem)




       UserID     ItemID Rating    Timestamp
1435   n1rk9i  강남구청역 MMN     5점  2021.04.21.
1436  1b2n7aq  강남구청역 MMN     1점  2021.02.25.
         ItemID img_url
1435  강남구청역 MMN    None
1436  강남구청역 MMN    None
         ItemID                                             review
1435  강남구청역 MMN                                               최고예요
1436  강남구청역 MMN  인스타그램에서 먹스타그램 계정 운영하는 리뷰어한이 입니다. 1. 사회적거리두기 방역...


In [16]:
rating_df.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lineSuin_cafe(final).csv')

In [17]:
User_ID_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lineSuin_cafe_User(final).csv')
item_img_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lineSuin_cafe_img(final).csv')
item_review_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/llineSuin_cafe_review(final).csv')

In [18]:
ItemID_elem = rating_df['ItemID'].unique()
len(ItemID_elem)


93

In [19]:
data.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lineSuin(final)url.csv')